In [1]:
import numpy as np
import pickle
import requests
import time

## Grab `tournament info`
Date, location, divisions, courses

In [2]:
# HTML headers to amicably communicate with the PDGA servers
headers = {'value': 'application/json, text/javascript, */*; q=0.01','accept': 'application/json, text/javascript, */*; q=0.01','cookie': 'cookie-agreed-version=1.0.1; clientSettings=%7B%22favoritePlayers%22%3A%5B%5D%2C%22showFavorites%22%3Atrue%2C%22metric%22%3Afalse%2C%22colorAccessibility%22%3Afalse%2C%22darkMode%22%3Atrue%2C%22throwTracker%22%3Atrue%2C%22userTimeZoneEnabled%22%3Atrue%7D'}

In [3]:
tournament_id = 89132
url = 'https://www.pdga.com/apps/tournament/live-api/live_results_fetch_event?TournID='

page = requests.get(url + str(tournament_id), headers=headers)

tournament_info = page.json()['data']

In [4]:
tournament_info

{'DateRange': 'Aug 9-10, 2025',
 'EndDate': '2025-08-10',
 'Country': 'United States',
 'Location': 'Perkasie, Pennsylvania',
 'LocationShort': 'Perkasie, PA',
 'Divisions': [{'DivisionID': 100,
   'Division': 'MPO',
   'DivisionName': 'Mixed Pro Open',
   'Players': 71,
   'IsPro': True,
   'ShortName': 'Mixed Pro Open',
   'AbbreviatedName': 'Mixed Pro Open',
   'LatestRound': 2},
  {'DivisionID': 105,
   'Division': 'FPO',
   'DivisionName': "Women's Pro Open",
   'Players': 14,
   'IsPro': True,
   'ShortName': "Women's Pro Open",
   'AbbreviatedName': "Women's Pro Open",
   'LatestRound': 2},
  {'DivisionID': 111,
   'Division': 'MP40',
   'DivisionName': 'Mixed Pro 40+',
   'Players': 21,
   'IsPro': True,
   'ShortName': 'Mixed Pro 40+',
   'AbbreviatedName': 'Mixed Pro 40+',
   'LatestRound': 2},
  {'DivisionID': 400,
   'Division': 'MA1',
   'DivisionName': 'Mixed Amateur 1',
   'Players': 38,
   'IsPro': False,
   'ShortName': 'Mixed Amateur 1',
   'AbbreviatedName': 'Mixed A

## Extract divisions from `tournament_info`
Will use subsequently to create player lists for each division

In [5]:
divisions = []
for div in tournament_info['Divisions']:
    divisions.append(div['Division'])

In [6]:
print(divisions)

['MPO', 'FPO', 'MP40', 'MA1', 'MA40', 'MA2', 'MA3']


## Grab `live_layout`

In [7]:
url = 'https://www.pdga.com/api/v1/live-tournaments/89132/live-layouts?include=LiveLayoutDetails'
live_layout = requests.get(url, headers=headers).json()

live_layout

[{'layoutId': 700303,
  'courseId': 25812,
  'courseName': 'Nockamixon State Park',
  'tournId': 89132,
  'name': 'LVO 2025',
  'holes': 18,
  'par': 68,
  'length': 9325,
  'units': 'Feet',
  'accuracy': 'M',
  'notes': '',
  'ssaRd1': 65.892,
  'ssaRd2': 64.391,
  'ssaRd3': None,
  'ssaRd4': None,
  'ssaRd5': None,
  'ssaRd6': None,
  'ssaRd7': None,
  'ssaRd8': None,
  'ssaRd9': None,
  'ssaRd10': None,
  'ssaSemis': None,
  'ssaFinals': None,
  'combinedSSA': None,
  'provisionalSSA': None,
  'challengeFactor': None,
  'updateDate': '2025-08-18T17:18:02+00:00',
  'liveLayoutDetails': [{'layoutDetailId': 18579574,
    'layoutId': 700303,
    'hole': 'H1',
    'holeOrdinal': 1,
    'label': '1',
    'tee': None,
    'target': None,
    'par': 5,
    'length': 845,
    'notes': 'Creek around green plays as river',
    'updateDate': '2025-08-06T14:01:06+00:00'},
   {'layoutDetailId': 18579575,
    'layoutId': 700303,
    'hole': 'H2',
    'holeOrdinal': 2,
    'label': '2',
    'tee': 

## Create list of `ResultID`'s
A player performance for the entire tournament is denoted by a `ResultID`.

In [8]:
# Create list of ResultID's
result_ids = []
baseurl = 'https://www.pdga.com/apps/tournament/live-api/live_results_fetch_updated_round_scores?TournID=89132&Division='

for div in divisions:
    url = baseurl + div
    page = requests.get(url, headers=headers)
    data = page.json()["data"]

    for d in data:
        result_ids.append(d['ResultID'])

print(result_ids)

[211586222, 211586679, 211653303, 211863789, 211586118, 211586517, 211633208, 211586319, 211586525, 211589754, 211940397, 211899475, 211878383, 211925757, 211587409, 212019900, 212018592, 211876645, 211792604, 211703637, 211898508, 211740116, 211586594, 211618411, 211899449, 211638413, 211587325, 212013095, 212008899, 211581461, 211581674, 212018626, 211951751, 211887814, 211588238, 211593542, 211948809, 211586579, 211949799, 211806823, 211586057, 211611952, 211688052, 211897410, 211688048, 211883536, 211916920, 212015146, 211587267, 211881568, 211757332, 211965179, 211589937, 211617032, 211992741, 211863790, 211951989, 212010853, 211866750, 211612614, 211587416, 211586065, 212025424, 211898980, 211759097, 211899141, 211819520, 212009717, 211744192, 211586584, 211587218, 212013544, 211777194, 212009503, 211706577, 211586629, 211775135, 211589718, 211709048, 211768110, 211651247, 211609878, 211808887, 211783723, 211953374, 211591334, 211752059, 211586986, 211831331, 211786279, 211916926

## Obtain scoring by fetching data for each `ResultID`

In [9]:
results = []

baseurl = 'https://www.pdga.com/apps/tournament/live-api/live_results_fetch_player?ResultID='
for i, id in enumerate(result_ids):
    print(f"Grabbing {i} of {len(result_ids)}.")
    page = requests.get(baseurl + str(id))
    data = page.json()['data']

    results.append(data)

    time.sleep(1)  # don't overload server

Grabbing 0 of 180.
Grabbing 1 of 180.
Grabbing 2 of 180.
Grabbing 3 of 180.
Grabbing 4 of 180.
Grabbing 5 of 180.
Grabbing 6 of 180.
Grabbing 7 of 180.
Grabbing 8 of 180.
Grabbing 9 of 180.
Grabbing 10 of 180.
Grabbing 11 of 180.
Grabbing 12 of 180.
Grabbing 13 of 180.
Grabbing 14 of 180.
Grabbing 15 of 180.
Grabbing 16 of 180.
Grabbing 17 of 180.
Grabbing 18 of 180.
Grabbing 19 of 180.
Grabbing 20 of 180.
Grabbing 21 of 180.
Grabbing 22 of 180.
Grabbing 23 of 180.
Grabbing 24 of 180.
Grabbing 25 of 180.
Grabbing 26 of 180.
Grabbing 27 of 180.
Grabbing 28 of 180.
Grabbing 29 of 180.
Grabbing 30 of 180.
Grabbing 31 of 180.
Grabbing 32 of 180.
Grabbing 33 of 180.
Grabbing 34 of 180.
Grabbing 35 of 180.
Grabbing 36 of 180.
Grabbing 37 of 180.
Grabbing 38 of 180.
Grabbing 39 of 180.
Grabbing 40 of 180.
Grabbing 41 of 180.
Grabbing 42 of 180.
Grabbing 43 of 180.
Grabbing 44 of 180.
Grabbing 45 of 180.
Grabbing 46 of 180.
Grabbing 47 of 180.
Grabbing 48 of 180.
Grabbing 49 of 180.
Grabbing 5

## Add information to `data` dictionary

In [10]:
data = {}

data["tournament_info"] = tournament_info
data["divisions"] = divisions
data["live_layout"] = live_layout
data["result_ids"] = result_ids
data["results"] = results

## Save `data` dictionary    

In [11]:
with open(f"../data/raw/{tournament_id}.pkl", 'wb') as f:
    pickle.dump(data, f) # serialize the list